## Introducción

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

## Importar

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

## Leer los datos

In [4]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head(8))
print()
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Se observa que el dataframe no contiene valores ausentes y que se compone de 5 columnas, de las cuales, la de principal interes es la columna 'is_ultra', la cual señala el plan actual contratado en el mes corriente. 

## Crear variables

In [5]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Se crean variables separadas para las caracteristicas y para el objetivo del modelo eliminando la columna 'is_ultra' del dataframe df. De este modo el modelo partira del conjunto de características para tratar de predecir el plan actualmente contratado.

## Segmentar los datos

In [6]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, random_state=12345)

Debido a que solicita separar los datos fuente en tres grupos: entrenamiento, validación y prueba, se utiliza dos veces de forma consecutiva el método train_test_split ya que por cada aplicación del método unicamente se obtienen la segmentación en dos grupos. De esta forma inicialmente se segmento el grupo de entrenamiento y el de prueba, indicando que el grupo de prueba contendría 20% de los datos por lo que el conjunto de entrenamiento almacenaría el 80%. La segunda segmentación toma el conjunto de entrenamiento resultante de la primer segmentación y lo vuelve a segmentar, de tal forma que el 25% de ese grupo es asignado al conjunto de validación, esto debido a que ese 25% corresponde al 20% del conjunto de datos fuente original. Obteniendo así una proporción de: 20% conjunto de prueba, 20% conjunto de validación y 60% conjunto de entrenamiento con respecto al 100% de los datos fuente originales, mismo que fue lo señalado en las lecciones como lo recomendable en este tipo de casos.

## Crear modelo, entrenar, definir hiperparámetros y seleccionar el mejor modelo

Se procede a crear dos modelos de clasificación, el árbol de decisión de clasificación y el bosque aleatorio de clasificación para comparar los modelos resultantes de Machine Learning. De esta forma se seleccionó al modelo que alcanzara la mayor exactitud. 

In [7]:
# Crear bosque
best_score = 0
best_est = 0
best_depth = 0

for est in range(10, 51, 10):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state = 12345, n_estimators= est, max_depth = depth)
        model.fit(features_train, target_train)
        score = model.score(features_valid, target_valid)
        print("La exactitud para los n_estimators de", est, ", depth=", depth, "es", score)
        if score > best_score:
            best_score = score
            best_est = est
            best_depth = depth
print()
print('La exactitud del mejor modelo de bosque aleatorio de clasificación en el conjunto de validación:', best_score, ', n_estimators :', best_est, ', best_depth :', best_depth)

La exactitud para los n_estimators de 10 , depth= 1 es 0.7309486780715396
La exactitud para los n_estimators de 10 , depth= 2 es 0.7636080870917574
La exactitud para los n_estimators de 10 , depth= 3 es 0.7713841368584758
La exactitud para los n_estimators de 10 , depth= 4 es 0.7776049766718507
La exactitud para los n_estimators de 10 , depth= 5 es 0.7776049766718507
La exactitud para los n_estimators de 10 , depth= 6 es 0.7853810264385692
La exactitud para los n_estimators de 10 , depth= 7 es 0.7869362363919129
La exactitud para los n_estimators de 10 , depth= 8 es 0.7853810264385692
La exactitud para los n_estimators de 10 , depth= 9 es 0.7947122861586314
La exactitud para los n_estimators de 10 , depth= 10 es 0.7900466562986003
La exactitud para los n_estimators de 20 , depth= 1 es 0.7325038880248833
La exactitud para los n_estimators de 20 , depth= 2 es 0.7620528771384136
La exactitud para los n_estimators de 20 , depth= 3 es 0.7713841368584758
La exactitud para los n_estimators de

In [8]:
# Crear árbol

best_score_tree = 0
best_depth_tree = 0

for depth_tree in range(1, 51):
    model_tree = DecisionTreeClassifier(random_state = 12345, max_depth = depth_tree)
    model_tree.fit(features_train, target_train)
    score_tree = model_tree.score(features_valid, target_valid)
    print("La exactitud para depth=", depth_tree, "es", score_tree)
    if score_tree > best_score_tree:
        best_score_tree = score_tree
        best_depth_tree = depth_tree
print()
print('La exactitud del mejor modelo de árbol de decisión de clasificación en el conjunto de validación:', best_score_tree, ', best_depth_tree :', best_depth_tree)

La exactitud para depth= 1 es 0.7387247278382582
La exactitud para depth= 2 es 0.7573872472783826
La exactitud para depth= 3 es 0.7651632970451011
La exactitud para depth= 4 es 0.7636080870917574
La exactitud para depth= 5 es 0.7589424572317263
La exactitud para depth= 6 es 0.7573872472783826
La exactitud para depth= 7 es 0.7744945567651633
La exactitud para depth= 8 es 0.7667185069984448
La exactitud para depth= 9 es 0.7620528771384136
La exactitud para depth= 10 es 0.7713841368584758
La exactitud para depth= 11 es 0.7589424572317263
La exactitud para depth= 12 es 0.7558320373250389
La exactitud para depth= 13 es 0.749611197511664
La exactitud para depth= 14 es 0.7573872472783826
La exactitud para depth= 15 es 0.7527216174183515
La exactitud para depth= 16 es 0.749611197511664
La exactitud para depth= 17 es 0.7387247278382582
La exactitud para depth= 18 es 0.7418351477449455
La exactitud para depth= 19 es 0.7356143079315708
La exactitud para depth= 20 es 0.7293934681181959
La exactitu

Comparando los resultados de ambos modelos se observa que en ambos casos la exactitud más alta alcanzada es superior al umbral de 0.75 de exactitud, sin embargo, el modelo de bosque aleatorio de clasificación tuvo una exactitud mayor con un valor de 0.7978 con el conjunto del dataset de validación mientras que el árbol de decisión de clasificación tuvo un máximo de exactitud de 0.7745. Esto es lo esperado ya que se menciona que el bosque aleatorio de clasificación es más exacto que el árbol, sin embargo requiere un mayor poder de computo. Algo que podemos observar es que para el caso del modelo de árbol, el máximo de exactitud se obtuvo con una profundidad de 7 nodos, y a partir de este punto comienza a disminuir considerablemente, lo cual podría deberse al sobreajuste que afecta a este tipo de modelos.
Con lo anteriormente mencionado se observó que utilizando un intervalo de estimadores de 0 a 50 que incrementan de 10 en 10 y a la vez abarcando una profundidad de árbol de decisión de 1 a 10 nodos, la mejor combinación observada es para el modelo del bosque aleatorio es aquella que utilizó 50 árboles con una profundidad de 10 en sus copas. A groso modo podemos observar como la exactitud de los modelos incrementa a la par que la profundidad de los modelos, al igual que también incrementa la exactitud al incrmentar el valor de estimadores.

## Usando el modelo sobre el conjunto de prueba

Utilizando el modelo de bosque aleatorio de clasificación se utiliza el modelo sobre el conjunto de datos de prueba.

In [9]:
final_model = RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=10) # inicializa el constructor para el modelo que tenía el mejor valor de RECM
final_model.fit(features_train, target_train) # entrenar el modelo en el conjunto de entrenamiento

predictions = final_model.predict(features_test)
score_final = final_model.score(features_test, target_test)
print('La exactitud de final_model en el conjunto de prueba es:', score_final )

La exactitud de final_model en el conjunto de prueba es: 0.7993779160186625


## Conclusion

Al aplicar el modelo final sobre el conjunto de datos de prueba se obtuvo una exactitud de 0.799 misma que es superior al umbral de 0.75 señalado como requerimiento para aceptar el modelo, por lo que se considera satisfactorio.